# Extract section headers for discharge summaries

This notebook investigates and tries to harmonize section headers for discharge summaries.

In [119]:
import pandas as pd
import numpy as np
import os
import re
import psycopg2
from IPython.display import display, HTML

Connect to the database.

In [5]:
# specify user/password/where the database is
sqluser = 'postgres'
sqlpass = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

query_schema = 'SET search_path to ' + schema_name + ';'

# connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

In [216]:
# load a single note
query = query_schema + """
select * from noteevents
where category = 'Discharge summary'
"""
df = pd.read_sql_query(query, con)
df.columns

Index(['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text'],
      dtype='object')

In [188]:
# read the map
smap = pd.read_csv('section-header-mapping.csv', header=0)
smap.columns = ['header','header_fixed']
smap = smap.set_index('header')['header_fixed'].to_dict()

In [198]:
# find section headers - outputs a list of matches
pattern = "\n[ ]*\n([A-z0-9 ]+)(:| WERE | INCLUD | IS | ARE)"

# summarize each section header found in the notes 
note_list = list()

# how many times it is preceeded by a blank line
# how many times it is all capital letters
# how many times it spans more than 1 line

def cleanup_header_name(header_name):
    # normalize header name: remove preceeding/trailing whitespace, convert to lower case
    return header_name.lstrip(' ').rstrip(' ').lower()

for i, row in df.iterrows():
    if row['description']=='Addendum':
        continue
    row_id = row['row_id']
    text = row['text']
    hadm_id = row['hadm_id']
    
    # remove bad matches
    match_list = list()
    for match in re.finditer(pattern, text, re.M):
        header_name = cleanup_header_name(match.groups()[0])
        if header_name in ('disp'):
            # not a true section header
            continue
        else:
            match_list.append(match)
    
    current_note_list = list()
    prev_match = None
    
    for match in match_list:
        header_name = cleanup_header_name(match.groups()[0])
        # initialize dictionary of meta-data for this section
        section_info_dict = {'first_match': False,
                             'blank_section': False,
                             'multiline_section': False,
                             'blank_line_preceeds': False,
                             'row_id': row_id,
                             'hadm_id': hadm_id,
                             'header': header_name,
                             'header_fixed': smap[header_name]}
                    
        if prev_match is not None:
            # checks that require the previous match object
            prev_section = text[prev_match.end():match.start()]

            # is the previous section blank
            if prev_section.replace('\n','').replace(' ','') == '':
                current_note_list[-1]['blank_section'] = True

            # does the previous section span more than 1 line (excluding the last, possibly new-line, character)
            if '\n' in prev_section[0:-1]:
                current_note_list[-1]['multiline_section'] = True

            # how many times it is preceeded by a blank line
            if text[match.start()-1] == '\n':
                section_info_dict['blank_line_preceeds'] = True
        else:
            section_info_dict['first_match'] = True

        prev_match = match
        current_note_list.append(section_info_dict)

    note_list.extend(current_note_list)
    
# info from section headers
sh = pd.DataFrame(note_list)

In [199]:
# total number of discharge summaries
N = (df['description']=='Report').sum()

grp = sh.groupby('header_fixed').sum()
grp.drop(['row_id','hadm_id'], axis=1, inplace=True)
grp = grp.merge(sh.groupby('header_fixed')[['row_id']].count(), how='left', left_index=True, right_index=True)
grp['percentage'] = grp['row_id']/N*100.0

#grp.sort_values('row_id', ascending=False).reset_index()[['header','header']].to_csv('section-header-mapping.csv', index=False)

grp.sort_values('row_id', ascending=False)

,blank_line_preceeds,blank_section,first_match,medication,multiline_section,row_id,percentage
header_fixed,,,,,,,
service,35.0,2872.0,7739.0,0.0,1949.0,53911,97.705566
past medical history,20501.0,30.0,348.0,77.0,50061.0,51233,92.852094
history of present illness,24501.0,21.0,380.0,3.0,49863.0,49891,90.419921
hospital course,20744.0,53.0,5.0,0.0,48608.0,48722,88.301285
admission medications,14556.0,39.0,0.0,47511.0,46038.0,47511,86.106530
allergies,296.0,59.0,0.0,4.0,40136.0,47254,85.640756
discharge condition,31424.0,31.0,3.0,0.0,40840.0,47191,85.526578
discharge disposition,32735.0,8.0,1.0,0.0,42524.0,46813,84.841510
physical examination,9004.0,78.0,3.0,0.0,46483.0,46580,84.419233


In [ ]:
# notes which do not have this section
hdr_chk = 'admission medications'
not_these_row_id = sh.loc[sh['header_fixed']==hdr_chk, 'row_id']
ex_notes = sh.loc[ (~np.in1d(sh['row_id'], not_these_row_id)) , :]

display( HTML( ex_notes.head().to_html() ) )

# text of the first one
row_id = ex_notes['row_id'].unique()[5]
print('Note for row_id {}'.format(row_id))
print(df.loc[df['row_id']==row_id, 'text'].values[0])

In [ ]:
# notes which have this section
hdr_chk = 'admission medications'
ex_notes = sh.loc[sh['header_fixed']==hdr_chk, :]
display( HTML( ex_notes.head().to_html() ) )

# text of a single note
row_id = ex_notes['row_id'].unique()[0]
print('Note for row_id {}'.format(row_id))
print(df.loc[df['row_id']==row_id, 'text'].values[0])